In [1]:
import libsedml
import libsbml
import itertools

In [2]:
doc = libsedml.SedDocument()
doc.setLevel(1)
doc.setVersion(5)

0

In [3]:
model_tnf = doc.createModel()
model_tnf.setId("model_tnf")
model_tnf.setSource("cell_fate.sbml")
model_tnf.setLanguage("urn:sedml:language:sbml")

0

In [4]:
# Turn TNF initial state to 1
change = model_tnf.createChangeAttribute()
change.setTarget("/sbml:sbml/sbml:model/qual:listOfQualitativeSpecies/qual:qualitativeSpecies[@qual:id='TNF']/@qual:initialLevel")
change.setNewValue("1")

0

In [5]:
simulation = doc.createUniformTimeCourse()
simulation.setId("simulation")
simulation.setInitialTime(0.0)
simulation.setOutputStartTime(0.0)
simulation.setOutputEndTime(100.0)
simulation.setNumberOfSteps(1000)

0

In [6]:
algo = simulation.createAlgorithm()
algo.setId("algo")
algo.setKisaoID("KISAO:0000581")

# thread_count
param = algo.createAlgorithmParameter()
param.setKisaoID("KISAO:0000529")
param.setValue("8")

# sample_count
param = algo.createAlgorithmParameter()
param.setKisaoID("KISAO:0000498")
param.setValue("10000")

# seed
param = algo.createAlgorithmParameter()
param.setKisaoID("KISAO:0000488")
param.setValue("100")

0

In [7]:
task_tnf = doc.createTask()
task_tnf.setId("task_tnf")
task_tnf.setModelReference("model_tnf")
task_tnf.setSimulationReference("simulation")

0

In [8]:
time = doc.createDataGenerator()
time.setId("data_time_tnf")
time.setName("time")
var_time = time.createVariable()
var_time.setId("t")
var_time.setName("time")
var_time.setTaskReference("task_tnf")
var_time.setSymbol("urn:sedml:symbol:time")
time.setMath(libsedml.parseFormula("t"))

# and one for S1
survival = doc.createDataGenerator()
survival.setId("data_survival_tnf")
survival.setName("Survival")
var_survival = survival.createVariable()
var_survival.setId("Survival")
var_survival.setName("Survival")
var_survival.setTaskReference("task_tnf")
var_survival.setTarget("/sbml:sbml/sbml:model/qual:listOfQualitativeSpecies/qual:qualitativeSpecies[@qual:id='Survival']")
survival.setMath(libsedml.parseFormula("Survival"))

nonacd = doc.createDataGenerator()
nonacd.setId("data_nonacd_tnf")
nonacd.setName("NonACD")
var_nonacd = nonacd.createVariable()
var_nonacd.setId("NonACD")
var_nonacd.setName("NonACD")
var_nonacd.setTaskReference("task_tnf")
var_nonacd.setTarget("/sbml:sbml/sbml:model/qual:listOfQualitativeSpecies/qual:qualitativeSpecies[@qual:id='NonACD']")
nonacd.setMath(libsedml.parseFormula("NonACD"))

apoptosis = doc.createDataGenerator()
apoptosis.setId("data_apoptosis_tnf")
apoptosis.setName("Apoptosis")
var_apoptosis = apoptosis.createVariable()
var_apoptosis.setId("Apoptosis")
var_apoptosis.setName("Apoptosis")
var_apoptosis.setTaskReference("task_tnf")
var_apoptosis.setTarget("/sbml:sbml/sbml:model/qual:listOfQualitativeSpecies/qual:qualitativeSpecies[@qual:id='Apoptosis']")
apoptosis.setMath(libsedml.parseFormula("Apoptosis"))

0

In [9]:
variables = ["Survival", "NonACD", "Apoptosis"]
states = {}
boolean_states = {}
for i in range(len(variables)+1):
    iter_states = itertools.combinations(variables, i)
    for state in iter_states:
        if len(state) == 0:
            str_state = "nil"
        else:
            str_state = " -- ".join(state)
        states.update({str_state: state})
        boolean_state = ""

        for var in variables:
            boolean_state += ("!" if not var in state else "") + var + " && "
        boolean_state = boolean_state[:-3]
        boolean_states.update({str_state: boolean_state})
print(states)
print(boolean_states)

{'nil': (), 'Survival': ('Survival',), 'NonACD': ('NonACD',), 'Apoptosis': ('Apoptosis',), 'Survival -- NonACD': ('Survival', 'NonACD'), 'Survival -- Apoptosis': ('Survival', 'Apoptosis'), 'NonACD -- Apoptosis': ('NonACD', 'Apoptosis'), 'Survival -- NonACD -- Apoptosis': ('Survival', 'NonACD', 'Apoptosis')}
{'nil': '!Survival && !NonACD && !Apoptosis ', 'Survival': 'Survival && !NonACD && !Apoptosis ', 'NonACD': '!Survival && NonACD && !Apoptosis ', 'Apoptosis': '!Survival && !NonACD && Apoptosis ', 'Survival -- NonACD': 'Survival && NonACD && !Apoptosis ', 'Survival -- Apoptosis': 'Survival && !NonACD && Apoptosis ', 'NonACD -- Apoptosis': '!Survival && NonACD && Apoptosis ', 'Survival -- NonACD -- Apoptosis': 'Survival && NonACD && Apoptosis '}


In [10]:
for name, t_variables in states.items():
    dg = doc.createDataGenerator()
    if len(t_variables) > 0:
        dg.setId("dg_" + ("_".join([t_var.lower() for t_var in t_variables])))
    else:
        dg.setId("dg_nil")
    dg.setName(name)
    for t_var in variables:
        var = dg.createVariable()
        var.setId(t_var)
        var.setName("t_var")
        var.setTaskReference("task_tnf")
        var.setTarget("/sbml:sbml/sbml:model/qual:listOfQualitativeSpecies/qual:qualitativeSpecies[@qual:id='%s']" % t_var)
    formula = libsedml.parseL3Formula(boolean_states[name])
    print("Checking formula for " + name + " = " + libsedml.formulaToL3String(formula))
    dg.setMath(formula)

Checking formula for nil = (!Survival) && (!NonACD) && !Apoptosis
Checking formula for Survival = Survival && (!NonACD) && !Apoptosis
Checking formula for NonACD = (!Survival) && NonACD && !Apoptosis
Checking formula for Apoptosis = (!Survival) && (!NonACD) && Apoptosis
Checking formula for Survival -- NonACD = Survival && NonACD && !Apoptosis
Checking formula for Survival -- Apoptosis = Survival && (!NonACD) && Apoptosis
Checking formula for NonACD -- Apoptosis = (!Survival) && NonACD && Apoptosis
Checking formula for Survival -- NonACD -- Apoptosis = Survival && NonACD && Apoptosis


In [11]:
# add a report
report = doc.createReport()
report.setId("report_tnf")
report.setName("report_tnf.csv")
set = report.createDataSet()
set.setId("ds1")
set.setLabel("time")
set.setDataReference("data_time_tnf")
set = report.createDataSet()
set.setId("ds2")
set.setLabel("Survival")
set.setDataReference("data_survival_tnf")
set = report.createDataSet()
set.setId("ds3")
set.setLabel("NonACD")
set.setDataReference("data_nonacd_tnf")
set = report.createDataSet()
set.setId("ds4")
set.setLabel("Apoptosis")
set.setDataReference("data_apoptosis_tnf")

0

In [12]:
# add a report
report = doc.createReport()
report.setId("report_states_tnf")
report.setName("report_states_tnf.csv")
set = report.createDataSet()
set.setId("states_time")
set.setLabel("time")
set.setDataReference("data_time_tnf")

for name, t_variables in states.items():

    if len(t_variables) > 0:
        dg_id = "dg_" + ("_".join([t_var.lower() for t_var in t_variables]))
    else:
        dg_id = "dg_nil"
    ds = report.createDataSet()
    ds.setId("ds_" + dg_id)
    ds.setLabel(name)
    ds.setDataReference(dg_id)


In [13]:
matplotlib_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
for i, color in enumerate(matplotlib_colors):
    style = doc.createStyle()
    style.setId("C%d" % i)
    line = style.createLineStyle()
    line.setColor(color)

style = doc.createStyle()
style.setId('line_green')
line = style.createLineStyle()
# line.setStyle('solid')
line.setColor('#2ca02cff')

style = doc.createStyle()
style.setId('line_red')
line = style.createLineStyle()
# line.setStyle('solid')
line.setColor('#d62728ff')

style = doc.createStyle()
style.setId('line_purple')
line = style.createLineStyle()
# line.setStyle('solid')
line.setColor('#9467bdff')

0

In [14]:
plot = doc.createPlot2D()
plot.setId('plot_trajectories_tnf')
plot.setLegend(True)

curve = plot.createCurve()
curve.setId('curve_survival_tnf')
curve.setName('Survival')
curve.setOrder(0)
curve.setXDataReference('data_time_tnf')
curve.setYDataReference('data_survival_tnf')
curve.setStyle('line_purple')

curve = plot.createCurve()
curve.setId('curve_apoptosis_tnf')
curve.setName('Apoptosis')
curve.setOrder(0)
curve.setXDataReference('data_time_tnf')
curve.setYDataReference('data_apoptosis_tnf')
curve.setStyle('line_green')

curve = plot.createCurve()
curve.setId('curve_nonacd_tnf')
curve.setName('NonACD')
curve.setOrder(0)
curve.setXDataReference('data_time_tnf')
curve.setYDataReference('data_nonacd_tnf')
curve.setStyle('line_red')

0

In [15]:
plot = doc.createPlot2D()
plot.setId('plot_states_trajectories_tnf')
plot.setLegend(True)

for i, (name, t_variables) in enumerate(states.items()):
    if len(t_variables) > 0:
        dg_id = "dg_" + ("_".join([t_var.lower() for t_var in t_variables]))
    else:
        dg_id = "dg_nil"
    curve = plot.createCurve()
    curve.setId('curve_' + dg_id)
    curve.setName(name if name != "nil" else "<nil>")
    curve.setOrder(i)
    curve.setXDataReference('data_time_tnf')
    curve.setYDataReference(dg_id)
    curve.setStyle("C%d" % i)

In [16]:
writer = libsedml.SedWriter()
writer.writeSedMLToFile(doc, "cell_fate_states.sedml")

True